### Dandelion Optimizer

#### Important Links:
- [Paper](https://www.sciencedirect.com/science/article/abs/pii/S0952197622002305)<br>
- [Matlab](https://www.mathworks.com/matlabcentral/fileexchange/114680-dandelion-optimizer)

#### Imports

In [1]:
using Dates

In [1]:
function initialization(Popsize, Dim, UB, LB)
    Boundary_no = length(UB) # number of boundaries
    
    # Preallocate array for Positions
    Positions = zeros(Popsize, Dim)

    # If the boundaries of all variables are equal and user enters a single number for both UB and LB
    if Boundary_no == 1
        Positions = rand(Popsize, Dim) .* (UB - LB) .+ LB
    end

    # If each variable has a different LB and UB
    if Boundary_no > 1
        for i in 1:Dim
            ub_i = UB[i]
            lb_i = LB[i]
            Positions[:, i] = rand(Popsize) .* (ub_i - lb_i) .+ lb_i
        end
    end

    return Positions
end

initialization (generic function with 1 method)

The above function initializes the starting positions of Dandelions.<br><br>
__Parameters__:
- Popsize: Determines the number of dandelions initialized
- Dim: States the number of dimensions of the problem
- UB, LB: This is an array giving the upper and lower bounds for the initialization values<br>

__Returns__:
- Initial Positions of all the dandelions

In [ ]:
function DO(Popsize, Maxiteration, LB, UB, Dim, Fobj)
    start_time = time()
    dandelions = initialization(Popsize, Dim, UB, LB)
    dandelionsFitness = zeros(Popsize)
    Convergence_curve = zeros(Maxiteration)
    for i in 1:Popsize
        dandelionsFitness[i] = Fobj(dandelions[i, :])
    end

    # Calculate the fitness values of initial dandelions.
    sorted_indexes = sortperm(dandelionsFitness)
    Best_position = dandelions[sorted_indexes[1], :]
    Best_fitness = dandelionsFitness[sorted_indexes[1]]
    Convergence_curve[1] = Best_fitness
    t = 2

	while t < Maxiteration + 1
    
		## Rising stage
		beta = randn(Popsize, Dim)
		alpha = rand() * ((1 / Maxiteration^2) * t^2 - 2 / Maxiteration * t + 1)  # eq.(8) in this paper
		a = 1 / (Maxiteration^2 - 2 * Maxiteration + 1)  # Note: U can use positive or negative on 1 for solving ur problem.
		b = -2a
		c = 1 - a - b
		k = 1 - rand() * (c + a * t^2 + b * t)  # eq.(11) in this paper
		if randn() < 1.5
			for i in 1:Popsize
				lamb = abs(randn(1, Dim))
				theta = (2 * rand() - 1) * pi
				row = 1 / exp(theta)
				vx = row * cos(theta)
				vy = row * sin(theta)
				NEW = rand(1, Dim) .* (UB .- LB) .+ LB
				dandelions_1[i, :] .= dandelions[i, :] .+ alpha .* vx .* vy .* lognpdf.(lamb, 0, 1) .* (NEW .- dandelions[i, :])  # eq.(5) in this paper
			end
		else
			for i in 1:Popsize
				dandelions_1[i, :] .= dandelions[i, :] .* k  # eq.(10) in this paper
			end
		end
		dandelions .= dandelions_1
		# Check boundaries
		dandelions .= max.(dandelions, LB)
		dandelions .= min.(dandelions, UB)
	